In [1]:
import os
from dotenv import load_dotenv

# Load environment variables (e.g., API keys for LLMs)
# Needs: pip install python-dotenv
load_dotenv() 

print("--- Loading Framework Components ---")

# Import necessary framework components
from sentientresearchagent.hierarchical_agent_framework.graph.task_graph import TaskGraph
from sentientresearchagent.hierarchical_agent_framework.graph.state_manager import StateManager
from sentientresearchagent.hierarchical_agent_framework.graph.execution_engine import ExecutionEngine
from sentientresearchagent.hierarchical_agent_framework.node.node_processor import NodeProcessor
from sentientresearchagent.hierarchical_agent_framework.context.knowledge_store import KnowledgeStore
# Assuming context_builder.py with resolve_context_for_agent exists
from sentientresearchagent. hierarchical_agent_framework.context.context_builder import resolve_context_for_agent 

# Import the agents package to trigger registration in agents/__init__.py
# This is crucial for populating AGENT_REGISTRY and NAMED_AGENTS
try:
    import sentientresearchagent.hierarchical_agent_framework.agents
    print("Imported agents package successfully (triggered registration).")
    # You could optionally check here if the registry was populated:
    from sentientresearchagent.hierarchical_agent_framework.agents.registry import AGENT_REGISTRY, NAMED_AGENTS
    if not AGENT_REGISTRY and not NAMED_AGENTS:
         print("\nWARNING: Agent registries appear empty after import. Check agents/__init__.py setup.\n")
    else:
         print(f"Agent Registry Keys: {list(AGENT_REGISTRY.keys())}")
         print(f"Named Agents: {list(NAMED_AGENTS.keys())}")

except ImportError as e:
    print(f"\nERROR: Failed to import agents package: {e}")
    print("Please ensure agents/__init__.py and agent definitions/adapters exist and are correct.")
    exit(1)
except Exception as e:
    print(f"\nERROR: An unexpected error occurred during agent import/setup: {e}")
    exit(1)


print("\n--- Initializing Framework ---")

--- Loading Framework Components ---
Defined: simple_test_planner_agno_agent
Executing agents/__init__.py: Setting up and registering agents...
AgentRegistry: Registered adapter 'PlannerAdapter' for action 'plan', task_type 'WRITE'
AgentRegistry: Registered adapter 'PlannerAdapter' for action 'plan', task_type 'THINK'
AgentRegistry: Registered adapter 'PlannerAdapter' for action 'plan', task_type 'SEARCH'
AgentRegistry: Registered adapter 'PlannerAdapter' with name 'default_planner'
AgentRegistry: Registered adapter 'PlannerAdapter' with name 'SimpleTestPlanner'
AGENT_REGISTRY populated: 3 entries.
NAMED_AGENTS populated: 2 entries.
Imported agents package successfully (triggered registration).
Agent Registry Keys: [('plan', <TaskType.WRITE: 'WRITE'>), ('plan', <TaskType.THINK: 'THINK'>), ('plan', <TaskType.SEARCH: 'SEARCH'>)]
Named Agents: ['default_planner', 'SimpleTestPlanner']

--- Initializing Framework ---


In [2]:
# Instantiate core components
task_graph = TaskGraph()
knowledge_store = KnowledgeStore()
state_manager = StateManager(task_graph)
node_processor = NodeProcessor() # NodeProcessor is currently stateless

# Instantiate the execution engine
execution_engine = ExecutionEngine(
    task_graph=task_graph,
    node_processor=node_processor,
    state_manager=state_manager,
    knowledge_store=knowledge_store
)

print("\n--- Starting Test Execution ---")

NodeProcessor initialized.

--- Starting Test Execution ---


In [3]:
# Define the root goal for the project
# project_goal = "Write a comprehensive report on the future of renewable energy."
project_goal = "Develop a short plan for writing a blog post about the benefits of remote work." # Simpler for testing

# Initialize the project with the root goal
# The 'default_planner' name should match one registered in agents/__init__.py
execution_engine.initialize_project(
    root_goal=project_goal,
    initial_agent_name="default_planner" 
)

TaskGraph: Added graph 'root_graph'. Is root: True
TaskGraph: Added node 'root' to graph 'root_graph'.
KnowledgeStore: Added/Updated record for root
CONSOLE LOG: Initialized with root node: root in graph root_graph


In [4]:
execution_engine.run_cycle(max_steps=10)

CONSOLE LOG: 
--- Starting Execution Cycle ---

CONSOLE LOG: --- Step 1 of 10 ---
Task root status updated to TaskStatus.READY. Result: None..., Error: None
KnowledgeStore: Added/Updated record for root
  Transition: Node root PENDING -> READY
  Processing Node: root (Status: READY, Layer: 0)
NodeProcessor: Received node root with status READY
  NodeProcessor: Handling READY node root (Type: PLAN, Goal: 'Develop a short plan for writi...')
Task root status updated to TaskStatus.RUNNING. Result: None..., Error: None
KnowledgeStore: Added/Updated record for root
ContextBuilder: Resolving context for task 'root' (Agent: default_planner, Type: WRITE)
ContextBuilder: No relevant context items found for task 'root'.
DEBUG: About to create AgentTaskInput. current_task_type variable value: 'WRITE'
AgentRegistry: Found adapter 'PlannerAdapter' by name 'default_planner' for node root.
    NodeProcessor: Invoking PLAN adapter 'PlannerAdapter' for root
  Adapter 'SimpleTestPlanner': Processing nod

NameError: name 'NodeType' is not defined